In [1]:
# This file contains the csv output for clustering resampled binary label data
# Still needs cleaning

## Expand a labeled dataset to SMOTE then Cluster

magnify the number of minority/exceptional cases within the sequence dataset, ideally targets the binary binned dataset.

[reference 1](<ver5-ordinal-binning-grid-searches/step 2-0, ranged clustering, with time.ipynb>)

different oversampling tools: Naive random oversampling, SMOTE, ADASYN, SMOTENC

In [2]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE,SMOTENC, KMeansSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from collections import Counter
import pandas as pd 

from sklearn.metrics import balanced_accuracy_score
from imblearn.ensemble import BalancedBaggingClassifier,BalancedRandomForestClassifier,RUSBoostClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, RandomForestClassifier, \
ExtraTreesClassifier, RandomTreesEmbedding, BaggingClassifier
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from scipy import stats

from sklearn.tree import DecisionTreeClassifier
#import shap
import pandas as pd
import time
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns
#import altair as alt

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  VotingClassifier 
from sklearn.model_selection import cross_val_score

from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score, silhouette_samples


In [3]:
m2_pipeline = pd.read_csv("binary_binned_pipeline.csv")

In [4]:
m2_pipeline.columns #do this to identify the index of the categorical feature, for below setup

Index(['Unnamed: 0', 'precursor_buy_cap_pct_change',
       'precursor_ask_cap_pct_change', 'precursor_bid_vol_pct_change',
       'precursor_ask_vol_pct_change', 'sum_change', 'length',
       'surge_targets_met_pct', 'time', 'label'],
      dtype='object')

In [5]:
keepable = ['precursor_buy_cap_pct_change', 
            'precursor_ask_cap_pct_change',
            'precursor_bid_vol_pct_change', 
            'precursor_ask_vol_pct_change',
            'sum_change','length','time']

In [6]:
y = m2_pipeline['label'] #per https://stackoverflow.com/a/73095562/12001832
X = m2_pipeline[keepable]
# Performing the test/train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#normalize all numeric columns
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
    

In [7]:
#set up toolsets as functions to build separate datasets, bin_Naive, bin_SMOTE, bin_ADASYN

def build_naive():  #https://imbalanced-learn.org/stable/over_sampling.html#naive-random-over-sampling
    ros = RandomOverSampler(random_state=42, sampling_strategy='minority')
    X_resampled, y_resampled = ros.fit_resample(X, y)
    #print("ROS",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_smote(): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html#smote
    X_resampled, y_resampled = SMOTE(random_state=42 ).fit_resample(X, y)
    #print("SMOTE",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_adasyn(): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.ADASYN.html#adasyn
    X_resampled, y_resampled = ADASYN(random_state=42,sampling_strategy='minority').fit_resample(X, y)
    #print("ADASYN",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_borderline(): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.BorderlineSMOTE.html#borderlinesmote
    X_resampled, y_resampled = BorderlineSMOTE(random_state=42,sampling_strategy='minority').fit_resample(X, y)
    #print("BORDERLINE",sorted(Counter(y_resampled).items())) 
    return X_resampled, y_resampled

def build_smotenc(): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTENC.html#smotenc
    smote_nc = SMOTENC( random_state=42,sampling_strategy='minority')
    X_resampled, y_resampled = smote_nc.fit_resample(X, y)
    #print("SMOTENC",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_svmsmote():
    sm = SVMSMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X, y)
    #print("SVMSMOTE",'Resampled dataset shape %s' % Counter(y_res))
    return X_res, y_res

def build_kmsmote(): #https://imbalanced-learn.org/stable/combine.html#combination-of-over-and-under-sampling
    m = KMeansSMOTE( random_state=42,sampling_strategy='minority')
    X_res, y_res = m.fit_resample(X, y)
    # Find the number of new samples in the middle blob
    n_res_in_middle = ((X_res[:, 0] > -5) & (X_res[:, 0] < 5)).sum()
    #print("KMSMOTE","Samples in the middle blob: %s" % n_res_in_middle)
    return X_res, y_res

def build_smoteenn():
    smote_enn = SMOTEENN(random_state=42,sampling_strategy='minority')
    X_resampled, y_resampled = smote_enn.fit_resample(X, y)
    #print("SMOTEENN",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_smotetomek():
    smote_tomek = SMOTETomek(random_state=42,sampling_strategy='minority')
    X_resampled, y_resampled = smote_tomek.fit_resample(X, y)
    #print("SMOTETOMEK",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

### take optimal classifier parameters and technique

In [8]:
# n_clusters = 2
def silhouette_scorer(estimator, X, y=None):
    labels = estimator.fit_predict(X)
    score = silhouette_score(X, labels)
    return score

In [9]:
samplers = [build_naive,build_smote,build_adasyn,build_borderline,build_svmsmote,build_smoteenn,build_smotetomek]
samplers_string = ["build_naive","build_smote","build_adasyn", "build_borderline", "build_svmsmote", "build_smoteenn","build_smotetomek"]
sampler_dict = {k: v for k, v in zip(samplers_string, samplers )}

n_clusters = [2,4,6,8,10,12]
data_list = []
for clus in n_clusters:
    for key, value in sampler_dict.items():
        X_sampled, y_sampled = value()
        #normalize all numeric columns
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_sampled)
        clf = KMeans(n_clusters =clus, init='k-means++', random_state = 42)
        labels = clf.fit_predict(X_train_scaled)

        df = pd.DataFrame({'y_sampled': y_sampled, 'Cluster_Label': labels})

        # Calculate the ratio of the larger count of labels (0 or 1) to the total count in each cluster
        cluster_ratios = df.groupby('Cluster_Label')['y_sampled'].value_counts().unstack().fillna(0)
        cluster_max_ratio = cluster_ratios.max(axis=1)
        cluster_total_count = cluster_ratios.sum(axis=1)
        cluster_profit_scores = cluster_max_ratio / cluster_total_count

        # Calculate the average profit score across clusters
        average_profit_score = np.mean(cluster_profit_scores)
        silhouette_score_1 = silhouette_score(X_train_scaled, labels)
        data_list.append({
            'clusters': clus,
            'model': key,
            'silhouette_score': silhouette_score_1,
            'profit_score': average_profit_score
        })
sampled_df = pd.DataFrame(data_list)

In [10]:
sampled_df.loc[sampled_df['silhouette_score'].idxmax()]

clusters                      2
model               build_smote
silhouette_score       0.793449
profit_score           0.715419
Name: 1, dtype: object

In [11]:
sampled_df.loc[sampled_df['profit_score'].idxmax()]

clusters                      12
model               build_adasyn
silhouette_score        0.263896
profit_score            0.762506
Name: 37, dtype: object

In [12]:
X_sampled, y_sampled = build_smote()
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_sampled)
clf = KMeans(n_clusters =2, init='k-means++', random_state = 42)
labels = clf.fit_predict(X_train_scaled)

In [13]:
# create final output csv containing most successful resampled data with cluster label 
df = pd.concat([X_sampled.reset_index(drop=True), pd.Series(labels, name='Cluster_Label')], axis=1)
df = pd.concat([df, y_sampled], axis =1)
df.to_csv("binary_clustered_resampled_pipeline.csv")